In [1]:
!docker-compose up -d --build

time="2025-12-06T17:35:47+07:00" level=warning msg="d:\\CEDT\\Course\\Data_Science\\project\\docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 Container traffy-zookeeper  Running
 Container traffy-redis  Running
 Container traffy-spark  Recreate
 Container traffy-kafka  Running
 Container traffy-spark  Recreated
 Container traffy-spark  Starting
 Container traffy-spark  Started


In [5]:
import pandas as pd
import redis
import json

csv_file_path = 'data/source/bangkok_traffy.csv' 

redis_key = 'traffy_staging_queue'

r = redis.Redis(host='localhost', port=6379, db=0)

In [6]:
# โหลดข้อมูลจากไฟล์ CSV เข้า Redis
# ข้อมูลทั้งหมดประมาณ 7.8 แสนแถว

BATCH_SIZE = 10000

try:
    total_rows = 0
    pipe = r.pipeline()

    for chunk in pd.read_csv(csv_file_path, chunksize=BATCH_SIZE, dtype=str):
        
        chunk = chunk.fillna("")
        for row in chunk.itertuples(index=False):
            row_dict = row._asdict()
            json_data = json.dumps(row_dict, ensure_ascii=False)
            pipe.lpush(redis_key, json_data)
        
        pipe.execute()
        
        total_rows += len(chunk)
        print(f" ⏳ ส่งแล้ว {total_rows} แถว...", end='\r')

    print(f"✅ เสร็จสิ้น! ส่งทั้งหมด {total_rows} แถว")

except Exception as e:
    print(f"❌ Error: {e}")


✅ เสร็จสิ้น! ส่งทั้งหมด 787026 แถว


In [5]:
data = r.lrange(redis_key, 0, 5)
for item in data:
    print(item.decode('utf-8'))

{"ticket_id": "2025-E7CTFD", "type": "{จราจร}", "organization": "เขตสวนหลวง,สำนักการจราจรและขนส่ง กรุงเทพมหานคร (สจส. กทม.)", "comment": "จริงๆไม่ใช่เขตสวนหลวงแต่ที่ปักเป็นสวนหลวงเพราะผมพักอาศัยอยู่ที่เขตนี้ สิ่งที่จะพูดถึงเส้นจราจรบนพื้นถนนในทั่วกทม. ไม่มีความเข้มไม่มีการสะท้อนในเวลากลางคืน จะออกกฏมากมายตั้งกล้องเพราะจับและปรับมากมาย แต่เส้นจาง บางมองแทบไม่เห็น ทุกๆเขตในกทม. ย้ำเลยว่าจางทุกๆที่ในกทม.", "photo": "https://storage.googleapis.com/traffy_public_bucket/attachment/2025-01/b90079901764000259df5fc4f23e177899cf915f.jpg", "photo_after": "", "coords": "100.65165,13.73036", "address": "2998 ถ. พัฒนาการ แขวงสวนหลวง เขตสวนหลวง กรุงเทพมหานคร 10250 ประเทศไทย", "subdistrict": "สวนหลวง", "district": "สวนหลวง", "province": "กรุงเทพมหานคร", "timestamp": "2025-01-16 02:53:34.290375+00", "state": "กำลังดำเนินการ", "star": "", "count_reopen": "0", "last_activity": "2025-01-16 02:58:02.568451+00"}
{"ticket_id": "EZCZZU", "type": "{ถนน}", "organization": "เขตจอมทอง", "comment": "ปัญหา: พบโรงกล

In [7]:
df = pd.read_csv(csv_file_path)

In [13]:
df.isna().sum()

ticket_id          8772
type                 97
organization        571
comment            8772
photo               115
photo_after      145717
coords                0
address            8772
subdistrict         566
district            561
province            195
timestamp             0
state                 0
star             512929
count_reopen          0
last_activity         0
dtype: int64